# Active work in LIFE Target Database

## starcat 4 stars that would be good to be in there

Jonah: golden targets

Felix: a lot fewer k and g stars in version 4 than version 3 catalog

In [ ]:
from utils.analysis import starcat_versions_lost_stars
lost_g_stars = starcat_versions_lost_stars()
print(lost_g_stars)



In [ ]:

# quite a number fail because of wds companions not having all the parameters we need -> would be solved by using dedicated multiplicity catalog for those. though some of those should have sptype in simbad, why do I not get that?
# - simbad object is not included... why not? create unit test for this to figure out
# some have issue that system name and primary name are the same so I get systems without children

In [3]:
# 1. have a first look at it

from utils.analysis import detail_info_object

starname='WDS J10114+4927B'#'*  11 LMi'
detail_info_object(database_tables,starname)
#wait, eta cas A is seen as system but does have stellar parameters...
#why is not having a parent a reason for exclusion?


Identifier 
       id            main_id      id_source_idref object_idref
---------------- ---------------- --------------- ------------
WDS J10114+4927B WDS J10114+4927B             197        18143

Object type
type
----
  st

 Basic stellar data
    main_id      class_lum class_temp class_temp_nr
---------------- --------- ---------- -------------
WDS J10114+4927B         ?          ?             ?

Parents
  parent_main_id   child_main_id   ... child_object_idref parent_object_idref
                                   ...                                       
----------------- ---------------- ... ------------------ -------------------
WDS J10114+4927AB WDS J10114+4927B ...              18143               18141

Siblings
  parent_main_id   child_main_id   ... child_object_idref parent_object_idref
                                   ...                                       
----------------- ---------------- ... ------------------ -------------------
WDS J10114+4927AB        HD 

In [ ]:
# 2. run the code specifically made to find out reason

import importlib
#refactured code: -> seems to be working
#now trying to print out and / or store as list exactly objects that lead to exclusion e.g. children
#hm I guess I would like to have a reference to which catalog 
#   I used to assicn the type of the object (gaia, wds, simbad)
import utils.analysis.catalog_comparison as cc
importlib.reload(cc)

# list given by jonah
l_golden_targets=['* alf Cen A','* alf Cen B','* alf Cen C','NAME Teegarden\'s Star','* tau Cet','* eps Eri','* eps Ind','HD  88230','* omi02 Eri','* sig Dra','HD 131977','* eta Cas A','* eta Cas B','TRAPPIST-1']
# list given by felix lost_g_stars

cc.detail_criteria(database_tables,list(lost_g_stars))

In [ ]:
from utils.analysis.catalog_comparison import detail_criteria
l_golden_targets=['* alf Cen A','* alf Cen B','* alf Cen C','NAME Teegarden\'s Star','* tau Cet','* eps Eri','* eps Ind','HD  88230','* omi02 Eri','* sig Dra','HD 131977','* eta Cas A','* eta Cas B','TRAPPIST-1']
detail_criteria(database_tables,l_golden_targets)
#to do, save star names without NAME in front.

### issue non physical multiples

### issue out of distance cut h_link


In [ ]:
distance_cut_in_pc = 30.
plx_in_mas_cut = 1000. / distance_cut_in_pc
print(plx_in_mas_cut)
#making cut a bit bigger for correct treatment of objects on boundary
plx_cut = plx_in_mas_cut - plx_in_mas_cut / 10.
print(plx_cut)
#something is wrong or is it just that 30 pc gives a strange result?, yes 30 pc is mathematically strange

In [ ]:
 # hm WDS J10114+4927B seems to have a way too small plx to be in our sample. so is just visual binary from wds
    # how do I incorporate this knowledge into my db? if I have this object in my db I should include its plx too
    # if I kick it out I should indicate that sibling is actually single star
    # ... maybe in life provider take the wds objects, fetch main_id, get simbad stuff for them,
    # run specific multiplicity catalog over them, adapt binary & h-link stuff
    # sounds more like something that should be done in wds provider.
    # right when creating the helpertable
    # like as soon as I have an id I should also query the plx
    # make a list of those whose plx are given but outside our cutoff
    # make a list of their associated objects
    # or right after running wds I take all objects of type sy and run a new provider with better
    # next idea:
    # at end of life provider (which should run last) get all possible plx and clean out objects that are too far out. give hosts and siblings special treatment e.g. a flag



In [ ]:
from provider.simbad import *
from astropy.table import Table


def test_wds_nonphysical_multiples():
    # data
    distance_cut_in_pc = 30.
    test_objects = ['WDS J10114+4927B']
    cat = Table(data=[['WDS J10114+4927B'],['HD 233714']],
                names=['id','main_id'],
                dtype=[object,object])
    # hm WDS J10114+4927B seems to have a way too small plx to be in our sample. so is just visual binary from wds
    # how do I incorporate this knowledge into my db? if I have this object in my db I should include its plx too
    # if I kick it out I should indicate that sibling is actually single star
    # ... maybe in life provider take the wds objects, fetch main_id, get simbad stuff for them,
    # run specific multiplicity catalog over them, adapt binary & h-link stuff

    # get main id of test objects
    fetch_main_id(cat, id_creator = IdentifierCreator(name='main_id', colname='id'))

    # function (problem, this is more of an integration test but the function
    #  is written in a way to enable easy exploration testing)
    sim_helptab, sim = create_simbad_helpertable(distance_cut_in_pc, test_objects)

    # assert

In [ ]:



#'* 212 Pup A' kicked out in non trivial binaries in starcat but in this code as trivial binary with companion not matching sptype
#['* zet01 Ret', '*  50 Per', 'HD  50281', 'HD  65907', '* eps Ind', 'V* V2215 Oph'] other way round

# database creation

In [ ]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

#provider_tables_dict, database_tables= ltd.partial_create(30,create=['life'])

provider_tables_dict, database_tables= ltd.create_life_td(30)

#issue: does not wokr with loading as then id_source_idref already exists -> put in a workaround with a warning. though now did not happen...

#issue: create not working right now because exomercat server not available -> put in a workaround
#issue: main_id missing in star_basic -> suddenly working??

#vizier not working? -> do tests first -> issue on topcat too -> solved

# load and analyze database

In [2]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

provider_tables_dict, database_tables= ltd.load_life_td()

#ah when it showed me an empty area in wds.py it actually didn't load it correctly. when I then cleaned it out it introduced an error. need to uncommit the minor fixes commit and then unstack that file

Loading life_td generated data
Loading sim data
Loading sdb data
Loading wds data
Loading exo data
Loading life data
Loading gaia data


# is this outdated?
Finding issue that causes M0-M5 stars to have too high temperatures (4250 instead of 3000). I think I solved this issue by adding quality assessment to gaia gspspec

In [ ]:
import numpy as np
#9=mesteff
#5=starbasic
#print(database_tables[5].colnames)
cols=['main_id','sptype_string','teff_st_qual','teff_st_value','teff_st_source_idref']
mstars=database_tables['star_basic'][cols][np.where(database_tables['star_basic']['class_temp']=='M')]
mstars_tempgiven=mstars[np.where(mstars['teff_st_value']!=1e+20)]
print(mstars_tempgiven[np.where(mstars_tempgiven['teff_st_value']>4000)])

In [ ]:
print(database_tables['sources'][np.where(database_tables['sources']['source_id']==874)])

In [ ]:
import numpy as np
#so lets find out if in gaia archive this is also the case
#first find gaia id
cols=['teff_st_qual','teff_st_value','teff_st_source_idref']

print(database_tables['mes_teff_st'][cols][np.where(database_tables['mes_teff_st']['main_id']=='HD 166348')])
#Gaia DR3 4848140361962951552

#on archive only gspphot showable and it is empty

In [ ]:
from sdata import empty_dict
from provider.utils import fetch_main_id, IdentifierCreator, create_provider_table, query
from pyvo.dal import TAPService

adql_query="""
    SELECT top 10 s.source_id ,p.mass_flame, p.radius_flame,
        p.teff_gspphot, p.teff_gspspec, m.nss_solution_type, p.age_flame
    FROM gaiadr3.gaia_source as s
        JOIN gaiadr3.astrophysical_parameters as p ON s.source_id=p.source_id
            LEFT JOIN gaiadr3.nss_two_body_orbit as m ON s.source_id=m.source_id
            WHERE s.source_id = 4848140361962951552"""
gaia = empty_dict.copy()
gaia['provider'] = create_provider_table('Gaia',
                                  "https://gea.esac.esa.int/tap-server/tap",
                                  '2016A&A...595A...1G')
try: 
    gaia_helptab=query(gaia['provider']['provider_url'][0],adql_query) 

except:
    #because of bug in gaia server where async not working currently
    service = TAPService(gaia['provider']['provider_url'][0])
    result=service.run_sync(adql_query.format(**locals()), maxrec=160000)
    gaia_helptab=result.to_table()


In [ ]:
print(gaia_helptab['source_id','teff_gspspec'])

In [ ]:
#so yes that is how it is in gaia. now find out if this parameter is reliable and if not lower its quality

In [ ]:
import utils.analysis as lta

lta.sanity_tests(database_tables,5)